# Documentacion 

Nosotros hemos elegido dos datasets sobre videojuegos, uno de ellos consta de la informacion de ventas de estos juegos mientras que los otros de la ubicacion de las empresas que los han creado. 

Como estos dos datasets eran del formato csv, hemos tenido que convertir uno de ellos a formato parquet. Para ellos hemos usado el siguiente codigo: 

In [ ]:
import pandas as pd

csv_file = "your_file.csv"
parquet_file = "your_file.parquet"

df = pd.read_csv(csv_file)

df.to_parquet(parquet_file, engine="pyarrow", index=False)

Con esto ya tendriamos guardado el dataset como formato parquet en el directorio en el que estamos. El siguiente paso seria el tratamiento de datos, donde vemos que datos son los que queremos mantener para luego juntar los datasets. 

Para ello nos dividimos los datasets, tratando uno cada uno, eliminado columnas innecesarias como `Est.` (El año en el que se establecio la empresa) que no son importantes para el objetivo que buscamos. Una vez hemos filtrado las columnas que nos interesan ahora queda tratar los valores nulos, en los que tenemos varias formas de elimianrlos, la que nosoros hemos usado es eliminar las filas con nulos, ya que al tener uan gran cantidad de datos y ser pocas filas podemos permitirnos perder algunas filas. 

Ahora con los datasets ya filtrados, podemos pasar a juntarlos, lo cual es algo simple. 



In [ ]:
video_games = pd.concat([indie_games_csv, video_games_csv], ignore_index=True)

Con esta funcion de pandas unimos los dos datasets, aunque, ahora nos surge otro problema, las filas que estan en un dataset pero no en el otro tienen valores nulos, los cuales debemos tratar de alguna forma. Ademas nos surgen algunos duplicados los cuales podemos eliminar con este codigo:

In [ ]:
video_games = video_games.drop_duplicates(subset='Developer', keep='first')

Ahora el unico problema que nos queda es con los valores nulos que surgen al mezclar este dataset, (TO BE CONTINUED)

## Hugging face [Link](https://huggingface.co/datasets/ItzRoBeerT/video-games-sales)

### Subir nuestro dataset a hugging face
Ya tenemos nuestro dataframe listo para subir a hugging face. 

Ahora sólo nos hace falta:
- Nuestro token de hugging (Disponible en access tokens)
- tener importado la librería `huggingface_hub` del cual usaremos la función `login`
- Importar la clase `Dataset`de la librería `datasets`

### Pasos a seguir:

1. Primero importamos las librerías que utilizaremos:
```python
from datasets import Dataset
from huggingface_hub import  login
```
2. Obtener nuestro token de acceso de hugging face:

Se puede obtener cuando hacemos click en  nuestro perfil -> access token -> create new token

!Importante conceder permisos de **escritura**¡

_OJO_
En este caso, hemos usado la función `getenv` de la librería `os` para tener nuestro token de acceso en una variable de entorno y que de esta forma, no sea accesible públicamente.

Sólo tenemos que llamar a la función `getenv` y pasarle el nombre que hemos especificado en nuestro archivo .env
```python

token = getenv('VARIABLE_CON_EL_TOKEN')
``` 

3. Parseamos nuestro dataframe a dataset 

Gracias a la clase `Dataset`, ya trae una función `from_pandas` la cual convierte nuestro dataframe en un dataset listo para publicarlo a nuestra cuenta de **huggingn face**

4. Loguearnos en hugging face

```python
login(token=token)
```

5. Publicar el dataset

```python
dataset.push_to_hub('nombre-del-repo')
```

## Aitor Mendrago [Link](https://aitor-medrano.github.io/iabd/hf/datasets.html)

### Crear un dataset a partir de nuestros datos

La forma más comun de crear un dataset a partir de nuestros propios datos, es crear un diccionario o una lista y crear el dataset a partir de ello. Luego podemos usar `Dataset.from_dict()` o `Dataset.from_list()` para crear nuestro dataset. 

In [1]:
# Primero instala la biblioteca si no la tienes
# pip install datasets

from datasets import Dataset

# Datos de ejemplo en formato diccionario (columnas)
datos_dict = {
    "texto": ["Hola mundo", "Python es genial", "Aprendiendo NLP"],
    "etiqueta": [0, 1, 0],
    "id": [1, 2, 3]
}

# Crear dataset desde diccionario
dataset_from_dict = Dataset.from_dict(datos_dict)

# Datos de ejemplo en formato lista (filas)
datos_list = [
    {"texto": "Hola mundo", "etiqueta": 0, "id": 1},
    {"texto": "Python es genial", "etiqueta": 1, "id": 2},
    {"texto": "Aprendiendo NLP", "etiqueta": 0, "id": 3}
]

# Crear dataset desde lista
dataset_from_list = Dataset.from_list(datos_list)

# Mostrar estructura del dataset
print("Dataset desde diccionario:")
print(dataset_from_dict)
print("\nPrimera fila:", dataset_from_dict[0])

print("\nDataset desde lista:")
print(dataset_from_list)
print("\nPrimera fila:", dataset_from_list[0])

Dataset desde diccionario:
Dataset({
    features: ['texto', 'etiqueta', 'id'],
    num_rows: 3
})

Primera fila: {'texto': 'Hola mundo', 'etiqueta': 0, 'id': 1}

Dataset desde lista:
Dataset({
    features: ['texto', 'etiqueta', 'id'],
    num_rows: 3
})

Primera fila: {'texto': 'Hola mundo', 'etiqueta': 0, 'id': 1}


### Limpiar el dataset

#### Trabajar con filas

El metodo que solemos usar más es el `Dataset.filter()` ya que nos permite buscar en el dataset en base a una condicion que le demos, aunque si queremos seleccionar solamente algunas filas podemos usar el metodo `Dataset.select()` para ello. 

In [14]:
from datasets import load_dataset

# Cargar un dataset de ejemplo
dataset = load_dataset("ag_news", split="train")

# Asegurarse de que la categoría (label) sea un entero
dataset = dataset.map(lambda example: {"label": int(example["label"])})

# Filtrar solo las filas donde la categoría (label) sea 2
filtered_dataset = dataset.filter(lambda example: example["label"] == 2)

# Seleccionar solo las columnas "text" y "label"
selected_dataset = filtered_dataset.remove_columns([col for col in filtered_dataset.column_names if col not in ["text", "label"]])

# Mostrar algunas filas del dataset resultante
print(selected_dataset[:5])

{'text': ["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.', 'Oil prices soar to all-time record, posing new menace to US econo

#### Trabajar con columnas

Para las columnas tenemos varias funciones, `Dataset.add_column()`, `Dataset.rename_column()` y `Dataset.remove_column()`, cada una hace lo que puedes esperar, añadir, renombrar y eliminar columnas. 

Otra funcion a destacar es la funcion `Dataset.map()`, la cual nos permite ejecutar una funcion en el dataset y si modificamos alguna columna del dataset que ya existe la sobreescribe. Esta funcion map() es la que usamos para tratar los datos del dataset, junto con la funcion `Dataset.sort()`, la cual es muy util para visualizar los datos 

In [2]:
from datasets import Dataset

# Dataset inicial
datos = {
    "texto": ["Transformers son poderosos", "Python vs Java", "IA revoluciona el mundo", "Hola"],
    "etiqueta": [1, 0, 1, 0],
    "id": [101, 102, 103, 104]
}

dataset = Dataset.from_dict(datos)

# Función para procesamiento con map
def procesar_ejemplo(ejemplo):
    # Calcular longitud del texto
    longitud = len(ejemplo["texto"])
    # Convertir texto a mayúsculas
    texto_mayus = ejemplo["texto"].upper()
    # Determinar si es positivo (etiqueta 1)
    es_positivo = ejemplo["etiqueta"] == 1
    return {
        "longitud": longitud,
        "texto_upper": texto_mayus,
        "es_positivo": es_positivo
    }

print("\nDataset inicial:", dataset)

# Aplicar map (agregando nuevas columnas)
dataset = dataset.map(procesar_ejemplo)

# Ordenar por longitud descendente
dataset = dataset.sort("longitud", reverse=True)

# Operaciones con columnas
# Renombrar columna
dataset = dataset.rename_column("etiqueta", "label")

# Agregar nueva columna calculada
dataset = dataset.add_column("palabras", [len(text.split()) for text in dataset["texto"]])

# Eliminar columna
dataset = dataset.remove_columns(["id"])

# Mostrar estructura final
print("\nDataset procesado:", dataset)

# Mostrar primeros 2 ejemplos
print("\nEjemplos finales:")
for ejemplo in dataset[:2]:
    print(ejemplo)


Dataset inicial: Dataset({
    features: ['texto', 'etiqueta', 'id'],
    num_rows: 4
})


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/4 [00:00<?, ? examples/s]


Dataset procesado:
Dataset({
    features: ['texto', 'label', 'longitud', 'texto_upper', 'es_positivo', 'palabras'],
    num_rows: 4
})

Ejemplos finales:
texto
label
longitud
texto_upper
es_positivo
palabras


La funcion map(), nos permite trabajar tambien con conjuntos de datos, seleccionando el parametro *batched* a True, de esta manera los datos se procesaran por conjuntos, es decir generando listas completas.

In [3]:
from datasets import Dataset

# Dataset con 3 textos
dataset = Dataset.from_dict({
    "texto": ["Hola", "Python es divertido", "Inteligencia Artificial"]
})

In [9]:
# Función que procesa UN solo ejemplo
def procesar_individual(ejemplo):
    print("Entrada de la funcion:", ejemplo)
    return {
        "longitud": len(ejemplo["texto"]),
        "mayusculas": ejemplo["texto"].upper()
    }

# Aplicar map (sin batched)
dataset_sin_batch = dataset.map(procesar_individual)

print("Sin batched (ejemplo por ejemplo):")
print(dataset_sin_batch[0])  # Primer ejemplo procesado

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Entrada de la funcion: {'texto': 'Hola'}
Entrada de la funcion: {'texto': 'Python es divertido'}
Entrada de la funcion: {'texto': 'Inteligencia Artificial'}
Sin batched (ejemplo por ejemplo):
{'texto': 'Hola', 'longitud': 4, 'mayusculas': 'HOLA'}


In [10]:
# Función que procesa un conjunto de ejemplos
def procesar_lote(lote):
    print("Entrada de la funcion", lote)
    return {
        "longitud": [len(t) for t in lote["texto"]],
        "mayusculas": [t.upper() for t in lote["texto"]]
    }

# Aplicar map (con batched=True)
dataset_con_batch = dataset.map(procesar_lote, batched=True)

print("\nCon batched (procesamiento por lotes):")
print(dataset_con_batch[0])  # Mismo resultado, pero procesado en bloque

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Entrada de la funcion {'texto': ['Hola', 'Python es divertido', 'Inteligencia Artificial']}

Con batched (procesamiento por lotes):
{'texto': 'Hola', 'longitud': 4, 'mayusculas': 'HOLA'}


#### Gestionar espacio de los datasets

Cuando tenemos datasets muy grandes, como por ejemplo BigCode que ocupa 67.5 TB,  que no podemos ejecutar en RAM podemos usar el parametro streaming de Dataset.load_dataset(), con este parametro accedemos a los datos por fragmenos, sobre los que podremos ir iterando usando la funcion next(), o con un bucle tradicional

In [11]:
from datasets import load_dataset

# Cargar dataset en modo streaming (aunque sea pequeño)
dataset = load_dataset("ag_news", split="train", streaming=True)

print("Primeros 3 ejemplos del dataset (streaming):")
for i, ejemplo in enumerate(dataset):
    if i >= 3:  # Limitar a 3 ejemplos para demostración
        break
    print(f"\nEjemplo {i + 1}:")
    print(f"Texto: {ejemplo['text'][:50]}...")  
    print(f"Etiqueta: {ejemplo['label']}")

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

Primeros 3 ejemplos del dataset (streaming):

Ejemplo 1:
Texto: Wall St. Bears Claw Back Into the Black (Reuters) ...
Etiqueta: 2

Ejemplo 2:
Texto: Carlyle Looks Toward Commercial Aerospace (Reuters...
Etiqueta: 2

Ejemplo 3:
Texto: Oil and Economy Cloud Stocks' Outlook (Reuters) Re...
Etiqueta: 2


#### Subir el dataset a Hugging Face

Para hacer el login en hugging face lo podemos hacer desde la terminal con `huggingface-cli login`, o con la funcion login() de hugging face hub. 

Para subir el datset podemos usar la funcion `push_to_hub`, y podemos cambiar el parametro private si querems subirlo en privado. Tambien tenemos el parametro split, por si tenemos datasets de train, y de validacion  

In [15]:
from huggingface_hub import login

# dentro de la funcion login metemos nuestro token de hugging face
login()

Token has not been saved to git credential helper.


In [ ]:
sample_dataset = Dataset.from_dict({
    "texto": ["Hola", "Python es divertido", "Inteligencia Artificial"]
})

sample_dataset.push_to_hub("user/repo_name")

Este es el enlace al dataset subido: [Enlace](https://huggingface.co/datasets/JuanjoJ55/video-games-sales)